## Считывание файла

In [4]:
import pyedflib
import numpy as np
import pandas as pd

In [2]:
def read_dataset(file_path):
    edf_file = pyedflib.EdfReader(file_path)
    n_signals = edf_file.signals_in_file
    signal_labels = edf_file.getSignalLabels()
    signals = [edf_file.readSignal(i) for i in range(n_signals)]
    edf_file.close()
    
    print('Сигналов обнаружено: ', n_signals)
    return signal_labels, signals

In [6]:
data_file_path = r"D:\Programming\international_hack_data\data\ECoG_unmarked [30 files, 6 h each]\Ati4x1_15m_BL_6h.edf"

signal_labels, signals = read_dataset(data_file_path)
signals = np.array(signals)
data = pd.DataFrame(signals).T.rename(columns={i: signal_labels[i] for i in range(len(signal_labels))})
data.head()

Сигналов обнаружено:  3


,FrL,FrR,OcR
0,0.050875,0.046562,0.121125
1,0.041813,0.036250,0.110250
2,0.051813,0.039938,0.122875
3,0.074750,0.067937,0.150938
4,0.086875,0.086063,0.177687


In [7]:
data.shape

(8640400, 3)

## Preprocessing

In [10]:
def prepare_sequences_for_prediction(new_data, seq_length=7):
    sequences = []
    for i in range(len(new_data) - seq_length + 1):
        sequences.append(new_data[i:i + seq_length])
    return np.array(sequences)

seq_length = 7
X_new_seq = prepare_sequences_for_prediction(data[:1000000], seq_length)

## Модель

In [11]:
# Загрузка модели
from tensorflow.keras.models import load_model
model = load_model('first_model.h5')

y_new_pred = model.predict(X_new_seq, batch_size=128)
most_probable_classes = np.argmax(y_new_pred, axis=1)
most_probable_classes.shape

7813/7813 [==============================] - 31s 4ms/step


(999994, 3)

In [22]:
most_probable_classes[most_probable_classes == 0].shape[0]

545131

In [23]:
most_probable_classes[most_probable_classes == 1].shape[0]

350707

In [24]:
most_probable_classes[most_probable_classes == 2].shape[0]

104156